# Laboratório 01 [ Estudo de Caso ] - Escolha do limiar de decisão para um problema de classificação.

## 1. __Introdução__

Este estudo de caso foca na análise e aplicação prática de um modelo de classificação existente para a detecção da COVID-19. A pandemia de COVID-19 evidenciou a necessidade crítica de diagnósticos rápidos e precisos para a contenção da propagação do vírus, principalmente para a população de grandes centros urbanos. Modelos de classificação baseados em inteligência artificial têm sido uma ferramenta valiosa para auxiliar no diagnóstico e triagem de pacientes. No entanto, um desafio significativo no uso desses modelos é o equilíbrio entre __sensibilidade__ (capacidade de identificar corretamente casos positivos) e __especificidade__ (capacidade de identificar corretamente casos negativos).

Um aspecto particular e crítico do problema é a __minimização de falsos negativos__ – casos em que pacientes com COVID-19 são erroneamente classificados como não tendo a doença. __Falsos negativos__ representam um risco significativo, não apenas para os indivíduos afetados, mas também para a saúde pública, pois pacientes não diagnosticados corretamente por um destes modelos pode contribuir para a propagação do vírus caso não seja tratado adequadamente.

__Lembrete:__ Você pode executar este laboratório de forma interativa utilizando o [Google Colab](https://colab.research.google.com/) através deste [link](https://colab.research.google.com/github/Secol/fiap-amd/blob/main/lab01/Lab01.EstudoDeCasoCovid19.ipynb).

![img](https://raw.githubusercontent.com/Secol/fiap-amd/main/lab01/assets/lab-banner-21.9.png)

## 2. __Definição do problema__

Neste laboratório iremos analisar as __saídas__ (probabilidades de ser positivo para COVID-19) de um modelo de classificação pré-existente e ajustar o __limiar de decisão__ para minimizar o número de falsos negativos. O objetivo é otimizar o modelo de forma que a probabilidade de um paciente verdadeiramente positivo ser __erroneamente classificado__ como negativo seja a menor possível.

O desafio consiste em encontrar o ponto ideal onde o modelo mantém uma alta sensibilidade sem comprometer excessivamente outras métricas importantes, como a especificidade e o valor preditivo negativo. Este equilíbrio é crucial para garantir que o modelo seja clinicamente útil e confiável na identificação de casos positivos, reduzindo assim o risco de propagação não detectada do vírus.

## 3. __Objetivos do laboratório__

- __Analisar as Probabilidades do Modelo:__ Estudar a distribuição das probabilidades de saída (classificação de positivo para COVID-19) fornecidas pelo modelo.
- __Ajustar o Limiar de Decisão:__ Identificar um limiar de decisão que minimize os falsos negativos, considerando o impacto nos falsos positivos e outras métricas relevantes.
- __Avaliar o Impacto do Ajuste do Limiar:__ Analisar como diferentes limiares afetam as métricas de desempenho do modelo, com ênfase na sensibilidade e especificidade.

Este laboratório visa não apenas explorar aspectos técnicos de modelagem de dados, mas também facilitar a colaboração efetiva de __arquitetos de solução__ ao apoio a times de analistas, cientistas e engenheiros na implementação de soluções baseadas em dados. Os objetivos específicos incluem:

1. Fornecer aos arquitetos de solução uma compreensão prática dos desafios e nuances da modelagem de dados, incluindo o equilíbrio entre diferentes métricas de desempenho como sensibilidade e especificidade.
2. Estabelecer um vocabulário comum para facilitar a comunicação eficaz entre as diferentes disciplinas envolvidas no projeto.
3. Demonstrar como as melhorias nos modelos podem impactar positivamente os resultados de negócios.
4. Inspirar ideias para desenvolver interfaces de usuário que apresentem os resultados do modelo de maneira compreensível e útil.

## 4. __Funções auxiliares__

In [1]:
# Importa a biblioteca 'requests' para permitir o envio de requisições HTTP em Python.
import requests

# Importa a biblioteca 'json' para trabalhar com dados no formato JSON.
import json

# Importa a biblioteca 'numpy' (apelidada de 'np') para manipulação de arrays e matrizes.
import numpy as np

# Importa o módulo 'graph_objects' da biblioteca 'plotly', utilizado para criar gráficos interativos e de alta qualidade.
import plotly.graph_objects as go

# Importa a função 'make_subplots' do módulo 'plotly.subplots', usada para criar layouts de gráficos que contêm múltiplos subplots.
from plotly.subplots import make_subplots

# Importa várias funções de 'sklearn.metrics', que são usadas para avaliar o desempenho de modelos de aprendizado de máquina:
# 'confusion_matrix' - Calcula a matriz de confusão para avaliar a precisão de uma classificação.
# 'classification_report' - Constrói um relatório de texto mostrando as principais métricas de classificação.
# 'roc_curve' - Calcula a curva ROC (Receiver Operating Characteristic), usada para visualizar o desempenho de modelos de classificação binária.
# 'auc' - Calcula a Área sob a Curva ROC, uma medida de desempenho do modelo.
# 'accuracy_score' - Calcula a precisão do modelo, a fração de amostras corretamente classificadas.
# 'f1_score' - Calcula o F1 score, uma medida de precisão do teste considerando tanto a precisão quanto a revocação.
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, accuracy_score, f1_score


In [2]:
# Define a função 'plot_distribuicao_base' que aceita 'y_real' como parâmetro.
# 'y_real' é um array que contém as classes reais dos dados (geralmente 0 para negativo e 1 para positivo).
def plot_distribuicao_base(y_real):

    # Calcula a quantidade de cada classe (0s e 1s) no array 'y_real'.
    contagens = np.bincount(y_real)
    total = len(y_real)
    percentagens = contagens / total * 100  # Calcula a porcentagem de cada classe.

    # Define os rótulos para as barras do gráfico.
    rotulos = ['Casos negativos', 'Casos positivos']

    # Define as cores para cada barra do gráfico.
    cores = ['#ed145b', '#00a2ff']

    # Cria um gráfico de barras usando Plotly.
    # Exibe as contagens e as porcentagens de cada classe.
    fig = go.Figure(data=[
        go.Bar(x=rotulos, y=contagens, text=np.round(percentagens, 2), textposition='auto', marker_color=cores)
    ])

    # Adiciona personalizações ao layout do gráfico, como título, rótulos dos eixos, espaço entre as barras e dimensões do gráfico.
    fig.update_layout(
        title='Distribuição de Classes em y_real',
        xaxis_title='Classe',
        yaxis_title='Quantidade',
        bargap=0.2,  # Define o espaço entre as barras.
        width=600, height=500  # Define as dimensões do gráfico.
    )

    # Exibe o gráfico.
    fig.show()

    # Imprime informações adicionais sobre o número total de amostras e a distribuição das classes.
    print(f'Número total de amostras: {total}')
    print(f'Número total de amostras negativas: {contagens[0]} ({contagens[0] / total * 100:.2f}%)')
    print(f'Número total de amostras positivas: {contagens[1]} ({contagens[1] / total * 100:.2f}%)')

# Define a função 'plot_histograma' que aceita 'dados_positivos', 'dados_negativos' e 'limiar' como parâmetros.
# 'dados_positivos' e 'dados_negativos' são arrays que contêm as probabilidades de saída do modelo para amostras positivas e negativas, respectivamente.
# 'limiar' é um valor numérico usado para destacar um ponto específico no histograma.
def plot_histograma(dados_positivos, dados_negativos, limiar):

    # Inicializa um objeto de figura Plotly.
    fig = go.Figure()

    # Adiciona um histograma para as amostras negativas ao gráfico.
    # 'marker_color' define a cor das barras e 'opacity' define a transparência.
    fig.add_trace(go.Histogram(x=dados_negativos, name='Amostras negativas', marker_color='#ed145b', opacity=0.75))

    # Adiciona um histograma para as amostras positivas ao gráfico.
    # Há um erro de digitação em 'Histogramb', deveria ser 'Histogram'.
    fig.add_trace(go.Histogram(x=dados_positivos, name='Amostras positivas', marker_color='#00a2ff', opacity=0.75))

    # Adiciona uma linha vertical no gráfico para indicar o limiar.
    # 'x0' e 'x1' definem a posição horizontal da linha, enquanto 'y0' e 'y1' definem a posição vertical (em coordenadas relativas).
    fig.add_shape(type='line', line=dict(dash='dash'), x0=limiar, x1=limiar, y0=0, y1=1, yref='paper')

    # Atualiza o layout do gráfico, definindo o modo de exibição das barras, título, rótulos dos eixos, tamanho e margens.
    fig.update_layout(barmode='overlay', title_text='Histograma das probabilidades de saíde de um modelo de classificação binária', xaxis_title='Valor de saída', yaxis_title='Frequência', autosize=False, width=800, height=500, margin=dict(l=50, r=50, b=100, t=100, pad=4))

    # Atualiza as configurações do eixo x, especificamente o ponto inicial dos ticks e a distância entre eles.
    fig.update_xaxes(tick0=0, dtick=0.1)

    # Exibe o gráfico.
    fig.show()

# Define a função 'analisa_limiar' que aceita 'intervalo_inicial' e 'intervalo_final' como parâmetros.
# No entanto, parece haver um erro, pois os parâmetros não são utilizados dentro da função.
def analisa_limiar(intervalo_inicial, intervalo_final, salto):

    # Itera sobre os limiares de 0 a 1, com incrementos de 0.1.
    # Isso é feito para avaliar o desempenho do modelo em diferentes pontos de corte para a classificação.
    for limiar in np.arange(intervalo_inicial, intervalo_final, salto):

        # Converte as probabilidades das previsões do modelo (y_modelo) em classificações binárias (0 ou 1),
        # com base no limiar atual. Valores acima do limiar são classificados como 1 (positivos),
        # e valores abaixo ou iguais ao limiar como 0 (negativos).
        y_preds = (y_modelo > limiar).astype(int)

        # Calcula a matriz de confusão comparando as previsões ajustadas (y_preds) com os valores reais (y_real).
        # A matriz de confusão é uma ferramenta que permite a visualização do desempenho de um algoritmo de classificação.
        # Ela fornece os valores de verdadeiros negativos (tn), falsos positivos (fp),
        # falsos negativos (fn) e verdadeiros positivos (tp).
        tn, fp, fn, tp = confusion_matrix(y_real, y_preds).ravel()

        # Calcula a Curva ROC (Receiver Operating Characteristic) e a Área Sob a Curva (AUC).
        # A Curva ROC é uma ferramenta gráfica utilizada para selecionar modelos de classificação binária
        # com base em sua performance. AUC representa a medida de separabilidade do modelo.
        fpr, tpr, thresholds = roc_curve(y_real, y_preds)
        roc_auc = auc(fpr, tpr)

        # Imprime os resultados da matriz de confusão e a AUC para cada limiar testado.
        # Isso permite uma análise detalhada do equilíbrio entre as taxas de verdadeiros positivos
        # e falsos positivos conforme o limiar é ajustado.
        print(f"Limiar {limiar:.2f}: TP: {tp:<5} | TN: {tn:<5} | FP: {fp:<5} | FN: {fn:<5} | roc_auc: {roc_auc*100:.2f}")

# Define a função 'show_results_parcial' com parâmetros para os dados, o modelo, o conjunto de teste e um limiar específico.
def show_results_parcial(dados_positivos, dados_negativos, y_modelo, y_real, limiar):
    # Calcula a matriz de confusão para as previsões do modelo com base no limiar especificado.
    y_preds = (y_modelo > limiar).astype(int)
    conf_matrix = confusion_matrix(y_real, y_preds)

    # Prepara textos para anotação na matriz de confusão.
    text = [[f'{conf_matrix[1][1]}', f'{conf_matrix[1][0]}'],
            [f'{conf_matrix[0][1]}', f'{conf_matrix[0][0]}']]

    # Define uma escala de cores personalizada para o gráfico (azul e vermelho).
    colorscale = [[0, '#00a2ff'], [1, '#ed145b']]

    # Cria um subplot com duas colunas: uma para o histograma e outra para a matriz de confusão.
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Probabilidades de Saída", "Matriz de Confusão"),
                        column_widths=[0.6, 0.4], horizontal_spacing=0.2)

    # Adiciona histogramas para dados positivos e negativos no primeiro subplot.
    fig.add_trace(go.Histogram(x=dados_negativos, opacity=0.75, name='Verdadeiros Negativos', marker_color='#ed145b'), row=1, col=1)
    fig.add_trace(go.Histogram(x=dados_positivos, opacity=0.75, name='Verdadeiros Positivos', marker_color='#00a2ff'), row=1, col=1)

    # Adiciona uma linha vertical no histograma para indicar o limiar.
    fig.add_shape(type='line', line=dict(dash='dash'), x0=limiar, x1=limiar, y0=0, y1=60, yref='paper', row=1, col=1)

    # Atualiza as legendas do eixo x do histograma.
    fig.update_xaxes(tick0=0, dtick=0.1)

    # Adiciona a matriz de confusão no segundo subplot como um heatmap.
    fig.add_trace(go.Heatmap(z=conf_matrix, x=['Positivo', 'Negativo'], y=['Positivo', 'Negativo'],
                            colorscale=colorscale, showscale=False, text=text, texttemplate="%{text}"), row=1, col=2)

    # Atualiza o layout geral do gráfico, incluindo títulos e configurações dos eixos.
    fig.update_layout(title_text='Análise do Modelo de Classificação', showlegend=False, barmode='overlay')
    fig.update_xaxes(title_text='Probabilidade', row=1, col=1)
    fig.update_yaxes(title_text='Contagem', row=1, col=1)
    fig.update_xaxes(title_text='Previsão', row=1, col=2)
    fig.update_yaxes(title_text='Real', row=1, col=2)

    # Exibe o gráfico.
    fig.show()

    # Imprime um relatório de classificação, fornecendo várias métricas de desempenho do modelo.
    print(classification_report(y_real, y_preds))

# Define a função 'show_results' que aceita os dados positivos, negativos, as previsões do modelo,
# os valores reais de teste e um limiar específico como parâmetros.
def show_results(dados_positivos, dados_negativos, y_modelo, y_test, limiar):
    # Calcula a matriz de confusão para as previsões do modelo com base no limiar especificado.
    y_preds = (y_modelo > limiar).astype(int)
    conf_matrix = confusion_matrix(y_test, y_preds)

    # Prepara textos para anotação na matriz de confusão.
    text = [[f'{conf_matrix[1][1]}', f'{conf_matrix[1][0]}'],
            [f'{conf_matrix[0][1]}', f'{conf_matrix[0][0]}']]

    # Calcula a Curva ROC e a Área Sob a Curva (AUC).
    fpr, tpr, thresholds = roc_curve(y_test, y_preds)
    roc_auc = auc(fpr, tpr)

    # Calcula a acurácia e o F1 Score do modelo.
    accuracy = accuracy_score(y_test, y_preds)
    f1 = f1_score(y_test, y_preds)

    # Cria um subplot com 2 linhas e 2 colunas para os gráficos.
    fig = make_subplots(rows=2, cols=2,
                        subplot_titles=("Probabilidades de Saída", "Curva ROC", "Matriz de Confusão"),
                        column_widths=[0.5, 0.5], row_heights=[0.6, 0.4], vertical_spacing=0.3)

    # Adiciona histogramas para dados positivos e negativos no primeiro subplot.
    fig.add_trace(go.Histogram(x=dados_negativos, opacity=0.75, name='Negativos', marker_color='#ed145b'), row=1, col=1)
    fig.add_trace(go.Histogram(x=dados_positivos, opacity=0.75, name='Positivos', marker_color='#00a2ff'), row=1, col=1)

    # Adiciona uma linha vertical no histograma para indicar o limiar.
    fig.add_shape(type='line', line=dict(dash='dash'), x0=limiar, x1=limiar, y0=0, y1=60, yref='paper', row=1, col=1)

    # Adiciona a Curva ROC no segundo subplot.
    fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=f'ROC curve (area = {roc_auc:.2f})'), row=1, col=2)
    fig.add_shape(type='line', line=dict(dash='dash'), x0=0, x1=1, y0=0, y1=1, row=1, col=2)

    # Adiciona a matriz de confusão no terceiro subplot como um heatmap.
    fig.add_trace(go.Heatmap(z=conf_matrix, x=['Positivo', 'Negativo'], y=['Positivo', 'Negativo'],
                             colorscale='Blues', showscale=False, text=text, texttemplate="%{text}"), row=2, col=1)

    # Atualiza o layout geral do gráfico, incluindo títulos e configurações dos eixos.
    fig.update_layout(title_text='Análise do Modelo de Classificação', showlegend=True, barmode='overlay')
    fig.update_xaxes(title_text='Probabilidade', row=1, col=1)
    fig.update_yaxes(title_text='Contagem', row=1, col=1)
    fig.update_xaxes(title_text='Taxa de Falso Positivo', row=1, col=2)
    fig.update_yaxes(title_text='Taxa de Verdadeiro Positivo', row=1, col=2)
    fig.update_xaxes(title_text='Previsão', row=2, col=1)
    fig.update_yaxes(title_text='Real', row=2, col=1)

    fig.update_layout(title_text='Análise do Modelo de Classificação', showlegend=True, barmode='overlay',
                      width=1200, height=800)  # Ajuste a largura e a altura conforme necessário

    # Mostrando o gráfico
    fig.show()

    print(classification_report(y_test, y_preds))

## 4. __Análise das amostras__

Para a realização deste laboratório, selecionei um conjunto de dados de saídas de um modelo de classificação elaborado para o problema. O primeiro passo envolverá o download dessas informações diretamente do GitHub, garantindo acesso aos dados necessários para nossa análise.

In [3]:
# URL do arquivo JSON
url = 'https://raw.githubusercontent.com/Secol/fiap-amd/main/lab01/data/dados.json'

# Baixar o conteúdo do arquivo JSON da web
response = requests.get(url)

# Verificar se a requisição foi bem-sucedida
if response.status_code == 200:
    # Carregar o conteúdo do JSON em um dicionário
    data = json.loads(response.text)
    print("JSON carregado com sucesso!")
else:
    print("Falha ao baixar o arquivo JSON. Status Code:", response.status_code)

JSON carregado com sucesso!


Para otimizar nosso processo de análise neste laboratório, vamos estruturar o conteúdo do arquivo JSON fornecido, atribuindo seus elementos a variáveis específicas. Isso nos permitirá acessar e manipular as informações de maneira mais eficiente:

In [4]:
# Importa a biblioteca NumPy para manipulação de arrays e operações matemáticas
import numpy as np

# Converte a lista ou coleção de dados positivos do modelo em um array NumPy
# 'data['dados_positivos']' deve conter os dados classificados como positivos
dados_positivos = np.array(data['dados_positivos'])

# Converte a lista ou coleção de dados negativos do modelo em um array NumPy
# 'data['dados_negativos']' deve conter os dados classificados como negativos
dados_negativos = np.array(data['dados_negativos'])

# Converte as previsões do modelo (saída do modelo) em um array NumPy
# 'data['y_modelo']' deve conter as previsões geradas pelo modelo de classificação
y_modelo = np.array(data['y_modelo'])

# Converte os valores reais (observados) em um array NumPy
# 'data['y_real']' deve conter os valores reais para comparação com as previsões do modelo
y_real = np.array(data['y_real'])

- `dados_positivos`: Lista contendo os valores (em probabilidade) de saída do modelo para os dados que são reais positivos.
- `dados_negativos`: Lista contendo os valores (em probabilidade) de saída do modelo para os dados que são reais negativos.
- `y_modelo`: Lista de classificações do modelo.
- `y_real`: Lista contendo as classificações reais, utilizada para validar as classificações do modelo.

Para obter uma compreensão mais clara da distribuição dos dados reais, podemos utilizar a função `plot_distribuicao_base`. Esta abordagem nos permitirá visualizar de forma gráfica e intuitiva como os dados estão distribuídos, facilitando a identificação de padrões ou discrepâncias relevantes.

In [5]:
plot_distribuicao_base(y_real)

Número total de amostras: 1000
Número total de amostras negativas: 808 (80.80%)
Número total de amostras positivas: 192 (19.20%)


Identificamos que a base de dados é desbalanceada, com uma predominância significativa de amostras negativas (80.8%) em comparação com as positivas (19.2%). Esse desequilíbrio pode ter implicações importantes na performance e na avaliação de modelos de classificação.

Bases desbalanceadas são comuns em muitos domínios, como detecção de fraude, diagnóstico médico e filtragem de spam, onde uma classe (geralmente a de maior interesse) é muito menos frequente do que a outra. Em tais cenários, a utilização de métricas tradicionais de avaliação, como acurácia, pode ser enganosa.

Por exemplo, em nosso caso, um modelo que simplesmente classifica todas as amostras como negativas atingiria uma acurácia de 80.8%, o que parece impressionante à primeira vista. No entanto, essa abordagem falha completamente em identificar as amostras positivas, que são cruciais. Este é um exemplo clássico de como a acurácia pode ser uma métrica enganosa em contextos de desequilíbrio de classe.

Portanto, é essencial escolher métricas mais adequadas que possam refletir efetivamente a performance do modelo em todas as classes. Métricas como a curva ROC-AUC, precisão, revocação e o F1-Score são mais informativas, pois levam em conta tanto a taxa de falsos positivos quanto a taxa de falsos negativos. A revocação é particularmente importante em muitos contextos onde detectar a classe minoritária (positiva, neste caso) é crucial.

## 5. __Análise das saídas do modelo__

Tendo em mãos o y_real, que representa as classificações reais das amostras, temos a oportunidade de investigar em detalhe as saídas probabilísticas geradas pelo modelo. Essa análise visa entender como as probabilidades atribuídas pelo modelo discriminam entre as diferentes classes reais. O foco está em explorar se e como as previsões do modelo se alinham ou divergem das classificações reais, proporcionando uma visão sobre a sua capacidade de distinguir acuradamente entre as categorias.

Através da função `plot_histograma` podemos plotar a visão:

In [6]:
plot_histograma(dados_positivos, dados_negativos, 0.5)

A função `show_results_parcial` não só nos permite examinar um histograma detalhado das probabilidades de saída, mas também fornece uma visão clara da matriz de confusão. Além disso, ela destaca as métricas-chave do modelo, facilitando uma compreensão mais profunda de sua eficácia e precisão na classificação dos dados.

In [7]:
show_results_parcial(dados_positivos, dados_negativos, y_modelo, y_real, 0.5)

              precision    recall  f1-score   support

           0       0.98      0.90      0.94       808
           1       0.68      0.93      0.79       192

    accuracy                           0.90      1000
   macro avg       0.83      0.91      0.86      1000
weighted avg       0.92      0.90      0.91      1000



- `macro avg (média macro)`: Calcula a média da precisão, revocação e F1-score sem levar em conta o suporte (proporção de cada classe). É útil quando se deseja tratar todas as classes igualmente.
- `weighted avg (média ponderada)`: Similar à média macro, mas aqui as métricas são ponderadas pelo suporte de cada classe. É útil quando as classes são desbalanceadas, como neste caso.

## 6. __Variando o valor do limiar__

Dada a importância crítica de assegurar que nenhum paciente verdadeiramente positivo para Covid-19 seja erroneamente classificado como um falso negativo pelo modelo, é essencial explorar diferentes valores de limiar. O objetivo é identificar um limiar ótimo que maximize a detecção correta de casos positivos, levando em consideração o desempenho atual do modelo. A função `analisa_limiar` nos permite conduzir uma série de testes avaliativos em uma gama de valores de limiar. Para isso, vamos iterar o valor do limiar de 0 a 1, aumentando-o em incrementos de 0.1 a cada passo, a fim de encontrar o equilíbrio ideal na classificação:

In [8]:
analisa_limiar(0, 1, 0.1)

Limiar 0.00: TP: 192   | TN: 14    | FP: 794   | FN: 0     | roc_auc: 50.87
Limiar 0.10: TP: 192   | TN: 67    | FP: 741   | FN: 0     | roc_auc: 54.15
Limiar 0.20: TP: 192   | TN: 193   | FP: 615   | FN: 0     | roc_auc: 61.94
Limiar 0.30: TP: 192   | TN: 394   | FP: 414   | FN: 0     | roc_auc: 74.38
Limiar 0.40: TP: 190   | TN: 608   | FP: 200   | FN: 2     | roc_auc: 87.10
Limiar 0.50: TP: 178   | TN: 726   | FP: 82    | FN: 14    | roc_auc: 91.28
Limiar 0.60: TP: 150   | TN: 787   | FP: 21    | FN: 42    | roc_auc: 87.76
Limiar 0.70: TP: 95    | TN: 802   | FP: 6     | FN: 97    | roc_auc: 74.37
Limiar 0.80: TP: 45    | TN: 808   | FP: 0     | FN: 147   | roc_auc: 61.72
Limiar 0.90: TP: 17    | TN: 808   | FP: 0     | FN: 175   | roc_auc: 54.43


Observamos que ao definir o limiar em 0.3, o número de falsos negativos é reduzido a zero. Contudo, ao ajustar o limiar para 0.4, o número de falsos negativos aumenta para 2. Essa variação sugere uma zona de transição crítica na sensibilidade do modelo. Para investigar mais precisamente este ponto de mudança, é aconselhável refazer a análise, desta vez estudando o intervalo entre 0.3 e 0.4. Propomos ajustar o limiar em incrementos mais finos de 0.01 para identificar o limiar ótimo que equilibra a detecção de casos positivos sem aumentar significativamente os falsos negativos.

In [9]:
analisa_limiar(0.3, 0.4, 0.01)

Limiar 0.30: TP: 192   | TN: 394   | FP: 414   | FN: 0     | roc_auc: 74.38
Limiar 0.31: TP: 192   | TN: 416   | FP: 392   | FN: 0     | roc_auc: 75.74
Limiar 0.32: TP: 192   | TN: 440   | FP: 368   | FN: 0     | roc_auc: 77.23
Limiar 0.33: TP: 192   | TN: 460   | FP: 348   | FN: 0     | roc_auc: 78.47
Limiar 0.34: TP: 192   | TN: 484   | FP: 324   | FN: 0     | roc_auc: 79.95
Limiar 0.35: TP: 192   | TN: 506   | FP: 302   | FN: 0     | roc_auc: 81.31
Limiar 0.36: TP: 192   | TN: 525   | FP: 283   | FN: 0     | roc_auc: 82.49
Limiar 0.37: TP: 192   | TN: 556   | FP: 252   | FN: 0     | roc_auc: 84.41
Limiar 0.38: TP: 190   | TN: 569   | FP: 239   | FN: 2     | roc_auc: 84.69
Limiar 0.39: TP: 190   | TN: 587   | FP: 221   | FN: 2     | roc_auc: 85.80
Limiar 0.40: TP: 190   | TN: 608   | FP: 200   | FN: 2     | roc_auc: 87.10


Após uma análise e considerando os diferentes valores de limiar, concluímos que o valor de 0.37 se destaca como o ponto ideal para o nosso modelo. Esta escolha do limiar em 0.37 representa um equilíbrio ótimo, maximizando a identificação correta de casos positivos de Covid-19, ao mesmo tempo em que mantém os falsos negativos em um mínimo aceitável. Portanto, 0.37 será o valor de limiar aplicado em nosso problema, visando a melhor performance na classificação de pacientes.

Podemos utilizar a função `show_results` para exibir o relatório das capacidades deste modelo considerando o limiar em 0.5 e o novo limiar ótimo em 0.37.

### Limiar = 0.5

In [10]:
show_results(dados_positivos, dados_negativos, y_modelo, y_real, 0.5)

              precision    recall  f1-score   support

           0       0.98      0.90      0.94       808
           1       0.68      0.93      0.79       192

    accuracy                           0.90      1000
   macro avg       0.83      0.91      0.86      1000
weighted avg       0.92      0.90      0.91      1000



### Limiar = 0.37

In [11]:
show_results(dados_positivos, dados_negativos, y_modelo, y_real, 0.37)

              precision    recall  f1-score   support

           0       1.00      0.69      0.82       808
           1       0.43      1.00      0.60       192

    accuracy                           0.75      1000
   macro avg       0.72      0.84      0.71      1000
weighted avg       0.89      0.75      0.77      1000



### 7. Conclusão

Ao finalizar nossa análise, observamos que a alteração do limiar de decisão do modelo de 0.5 para 0.37 resultou em algumas mudanças nas métricas de desempenho geral. Embora tenha havido uma ligeira redução nas capacidades globais do modelo, esta decisão estratégica cumpriu um objetivo crucial de negócio: garantir que nenhum paciente verdadeiramente positivo para Covid-19 fosse classificado erroneamente como um falso negativo.

Ao adotar o limiar de 0.37, nosso modelo alinha-se perfeitamente com a prioridade mais significativa do negócio - a segurança e a saúde dos pacientes. Isso reflete um entendimento profundo de que, em certas circunstâncias, a precisão global pode ser sacrificada em favor de uma métrica mais crítica, neste caso, a eliminação de falsos negativos na detecção da Covid-19.

Portanto, embora a troca do limiar possa ter impactado algumas métricas de desempenho, ela fortaleceu o modelo em um aspecto vital que transcende números: a confiabilidade e a segurança no diagnóstico de uma condição de saúde crítica.